In [1]:
# 1. Import thư viện
import pandas as pd
from datetime import datetime

# 2. Đọc dữ liệu từ file CSV
file_path = "online_sales_dataset.csv"
df = pd.read_csv(file_path)

In [2]:
# 3. Loại bỏ các hàng trùng lặp
df.drop_duplicates(inplace=True)

# 4. Loại bỏ các hàng có giá trị rỗng
df.dropna(inplace=True)

# 5. Xoá cột SalesChannel nếu tồn tại
if 'SalesChannel' in df.columns:
    df.drop(columns=['SalesChannel'], inplace=True)

# 6. Định dạng cột ngày tháng (InvoiceDate) về dạng "DD/MM/YYYY HH:MM:SS AM/PM"
def standardize_date(date_str):
    # Định dạng lại các ký hiệu SA/CH thành AM/PM
    date_str = date_str.replace(" SA", " AM").replace(" CH", " PM")
    formats = [
        '%d/%m/%Y %I:%M %p',  # Định dạng AM/PM
        '%d/%m/%Y %H:%M',     # Định dạng 24h
        '%Y-%m-%d %H:%M',     # Định dạng ISO 24h
        '%d/%m/%Y %H:%M %p'   # Định dạng lộn xộn SA/CH
    ]
    for fmt in formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime('%d/%m/%Y %I:%M:%S %p')
        except ValueError:
            continue
    # Nếu không phù hợp định dạng nào, trả về giá trị gốc
    return date_str

df['InvoiceDate'] = df['InvoiceDate'].apply(standardize_date)

# 7. Loại bỏ các hàng có giá trị âm hoặc không hợp lệ
# - Loại bỏ hàng có Quantity <= 0 và UnitPrice <= 0
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# - Loại bỏ các hàng có ShippingCost âm hoặc quá cao (outlier)
df = df[df['ShippingCost'].between(0, 1000)]

# - Loại bỏ các hàng có Discount ngoài khoảng 0-1
df = df[df['Discount'].between(0, 1)]

# 8. Sửa danh mục sản phẩm không đúng bằng bảng ánh xạ
# Bảng ánh xạ giữa Description và Category
description_category_map = {
    "White Mug": "Accessories",
    "Headphones": "Electronics",
    "Desk Lamp": "Electronics",
    "USB Cable": "Electronics",
    "Office Chair": "Furniture",
    "Notebook": "Stationery",
    "Wireless Mouse": "Electronics",
    "Blue Pen": "Stationery",
    "Wall Clock": "Accessories",
    "T-shirt": "Apparel",
    "Backpack": "Apparel"
}

# Hàm cập nhật Category theo Description
def update_category(row):
    description = row['Description']
    if description in description_category_map:
        return description_category_map[description]
    return row['Category']

df['Category'] = df.apply(update_category, axis=1)

# 9. Viết hoa chữ cái đầu của "paypall" trong cột PaymentMethod
df['PaymentMethod'] = df['PaymentMethod'].replace('paypall', 'Paypall')

In [3]:
# 10. Xuất dữ liệu đã tiền xử lý ra file CSV mới
output_path = "online_sales_dataset_cleaned_v2.csv"
df.to_csv(output_path, index=False)

print("Tiền xử lý dữ liệu hoàn tất! Dữ liệu đã được lưu tại:", output_path)

Tiền xử lý dữ liệu hoàn tất! Dữ liệu đã được lưu tại: online_sales_dataset_cleaned_v2.csv
